# 9장 불균형 범주의 문제

> 불균형 범주의 문제를 다룹니다.
- author: "Chansung Park"
- toc: true
- comments: true
- permalink: /chapter9/
- badges: true
- sticky_rank: 9

In [ ]:
#hide

!pip install fastai==1.0.61
!pip install yellowbrick==0.9
!pip install scikit-learn==0.21.0
!pip install pyjanitor==0.16.6
!pip install category_encoders==2.0.0
!pip install rfpimp
!pip install pandas==0.23.4

     |████████████████████████████████| 286kB 5.3MB/s 
     |████████████████████████████████| 12.8MB 6.1MB/s 
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: yellowbrick 0.9.1
    Uninstalling yellowbrick-0.9.1:
      Successfully uninstalled yellowbrick-0.9.1


     |████████████████████████████████| 6.7MB 5.2MB/s 
ERROR: umap-learn 0.5.1 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.0 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 2.3MB 6.7MB/s 
  Created wheel for sphinxcontrib-fulltoc: filename=sphinxcontrib_fulltoc-1.2.0-py2.py3-none-any.whl size=8715 sha256=79ab92c90e8c0ec80bcc4e32dfebd718542ea2e7ca5da23853136454a5f9768c
  Stored in directory: /root/.cache/pip/wheels/74/30/b2/f2da9acb18f37b571f3c17aa03ee6f1ac555514d99d76b9af0
Successfully built sphinxcontrib-fulltoc


     |████████████████████████████████| 92kB 4.0MB/s 
  Created wheel for rfpimp: filename=rfpimp-1.3.7-cp37-none-any.whl size=10670 sha256=c9747e729cd7da7288613535b14c270069bbaa6f48a69b48e9e68a310496930d
  Stored in directory: /root/.cache/pip/wheels/16/08/23/9d90df482c9c943df4d6fe874c0937f8e81dc3db917c9300c2
Successfully built rfpimp
     |████████████████████████████████| 8.8MB 5.6MB/s 
ERROR: xarray 0.15.1 has requirement pandas>=0.25, but you'll have pandas 0.23.4 which is incompatible.
ERROR: pyjanitor 0.16.6 has requirement pandas>=0.24.0, but you'll have pandas 0.23.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: plotnine 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.23.4 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: mizani 0.6.0 has requirement pandas>=0.25.0, but you'll 

In [ ]:
#hide

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#hide

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (
  ensemble,
  preprocessing,
  tree
)
from sklearn.metrics import (
  auc,
  confusion_matrix,
  roc_auc_score,
  roc_curve
)
from sklearn.model_selection import (
  train_test_split,
  StratifiedKFold
)
from yellowbrick.classifier import (
  ConfusionMatrix,
  ROCAUC
)
from yellowbrick.model_selection import (
  LearningCurve
)

from sklearn import impute

In [ ]:
#hide

url = "https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls"
df = pd.read_excel(url)
orig_df = df

df = pd.get_dummies(df)
df = df.drop(columns="sex_male")
df = pd.get_dummies(df, drop_first=True)

y = df.survived
X = df.drop(columns="survived")

def tweak_titanic(df):
  df = df.drop(
    columns=[
      "name",
      "ticket",
      "home.dest",
      "boat",
      "body",
      "cabin",
    ]
  ).pipe(pd.get_dummies, drop_first=True)
  return df

def get_train_test_X_y(df, y_col, size=0.3, std_cols=None):
  y = df[y_col]
  X = df.drop(columns=y_col)
  X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=size, random_state=42
  )
  cols = X.columns
  num_cols = [
    "pclass",
    "age",
    "sibsp",
    "parch",
    "fare",
  ]

  fi = impute.IterativeImputer()

  X_train.loc[:, num_cols] = fi.fit_transform(X_train[num_cols])
  X_test.loc[:, num_cols] = fi.transform(X_test[num_cols])

  if std_cols:
    std = preprocessing.StandardScaler()
    X_train.loc[:, std_cols] = std.fit_transform(X_train[std_cols])
    X_test.loc[:, std_cols] = std.transform(X_test[std_cols])

  return X_train, X_test, y_train, y_test  

ti_df = tweak_titanic(orig_df)
std_cols = "pclass,age,sibsp,fare".split(",")

X_train, X_test, y_train, y_test = get_train_test_X_y(ti_df, "survived", std_cols=std_cols)  

from sklearn.dummy import DummyClassifier
bm = DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test) # 정확도

from sklearn import metrics
metrics.precision_score(y_test, bm.predict(X_test))

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

# 9.4 소수집단 데이터 업샘플링하기

In [ ]:
from sklearn.utils import resample
mask = df.survived == 1
surv_df = df[mask]
death_df = df[~mask]

df_upsample = resample(
  surv_df,
  replace=True,
  n_samples=len(death_df),
  random_state=42,
)

df2 = pd.concat([death_df, df_upsample])
df2.survived.value_counts()

1    809
0    809
Name: survived, dtype: int64

In [ ]:
from imblearn.over_sampling import (
  RandomOverSampler,
)

ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_sample(X, y)
pd.Series(y_ros).value_counts()

1    809
0    809
dtype: int64

# 9.5 과반수집단 데이터를 다운샘플링하기

In [ ]:
from sklearn.utils import resample
mask = df.survived == 1
surv_df = df[mask]
death_df = df[~mask]

df_downsample = resample(
  death_df,
  replace=False,
  n_samples=len(surv_df),
  random_state=42,
)

df3 = pd.concat([surv_df, df_downsample])
df3.survived.value_counts()

1    500
0    500
Name: survived, dtype: int64